In [1]:
#node for code tests
from utils import *
import threading
import time

# debug_hap()
hap = initialize_hap(port = "COM3")
if (hap is None):
    raise 

hap_action(hap)
socket,client,addr = initialize_al()
rob = initialize_robot()

# # # # rough calibration
al_2_robot_tf = get_al_2_robot_tf()

# if not (client is not None and rob is not None and hap is not None):
#     raise

hap is connected
hap function executed
socket created 
socket binded to port 
socket listensing ... 


tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 69
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1093
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1375


[0.0298709  0.01104581 0.26404074 1.        ]


In [2]:
hap_action(hap)

hap function executed


In [3]:
import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
from IPython.display import clear_output
# from _thread import *
import threading
import csv
from csv import writer

tag = []
state = []
device_position_r = []
velocity = []
lock = threading.Lock()

def process_data_from_AL():
    global tag,state,device_position_r,velocity,lock
    print("thread has started")
    while True :         
        
        incoming_message = client.recv(1024).decode("utf-8") #Collect data from port and decode into  string
        data_start_ix = incoming_message.find("[")
        data_end_ix = incoming_message.find("]")
        data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)

        while(not data_found):
            incoming_message = client.recv(1024).decode("utf-8") #Collect data from port and decode into  string
            data_start_ix = incoming_message.find("[")
            data_end_ix = incoming_message.find("]")
            data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)
        
        incoming_message = incoming_message[data_start_ix+1 : data_end_ix] 
        incoming_message = incoming_message.replace(',','.')
        list_matrix = incoming_message.split(";")
        lock.acquire()
        state,position,velocity = (list_matrix[1],
                                     np.fromstring(list_matrix[2], dtype=np.float,sep =" "),
                                     np.fromstring(list_matrix[3], dtype=np.float,sep =" "))
        #swap_z 
        position[2] *= -1 
        position_al = np.hstack((position,1))
        device_position_r = al_2_robot_tf @ position_al
        lock.release()

def check_distance_ee_al(hap_act_dist = 0.35,
                         react_vel_threshold = 0.1,
                         critical_dist = 0.1):
    time.sleep(2)
    global writer_active,distance_triggered,error_triggered,reaction_triggered,activate_haptic
    start_time = time.time_ns()
    while True:
#         if(device_position_r.shape[0] < 3): continue
        distance = np.linalg.norm(np.array(device_position_r[:3]) - np.array(rob.getl())[:3])
        velocity_magnitude = np.linalg.norm(velocity)
        if (writer_active):
            write_data(writer_all_data, distance, velocity_magnitude, device_position_r,time.time_ns())   
            
            if(not distance_triggered and distance < hap_act_dist):
                start_time = time.time_ns()
                event = "Hand entered haptic area"
                distance_triggered = True
                activate_haptic = True
                
                write_data(writer_events,event,start_time)
                print(event, distance)
            
            if(velocity_magnitude > react_vel_threshold and not distance_triggered and not error_triggered):
                event = "User has moved to early"
                write_data(writer_events,event,time.time_ns()) 
                error_triggered = True
            #calc user reaction time 
            
            if(velocity_magnitude > react_vel_threshold and distance_triggered and not reaction_triggered):
                event = "User has reacted"
                reaction_time = (time.time_ns() - start_time)/ 1e9
                print(event, reaction_time)
                reaction_triggered = True
                write_data(writer_events,event, reaction_time)
            
            if(distance < critical_dist):
                event = "User has reached critical distance"
                write_data(writer_events,elapsed_time)
                print(event, distance)

def haptic_thread():
    global activate_haptic,hap
    while True:        
        if(activate_haptic):
            print("trying hap action")
            hap_action(hap)
            activate_haptic = False
        time.sleep(0.01) 
        
def write_data(writer,*data):
    writer.writerow(data)

a = 0.05
v = 0.1 # m/s

safe_distance_to_hand = 0.3 # meters
hap_act_dist = 0.4  # meters
react_vel_threshold = 0.1

user_attempts = 2
user_name = 'Viktor_2'

# we want to conveniently store the data
writer_active = False 

# user error,reaction time are calculated only once per attempt
distance_triggered = False
error_triggered = False 
reaction_triggered = False
activate_haptic = False

data_from_al_thread = threading.Thread(target = process_data_from_AL, daemon = True, args=())
data_from_al_thread.start()

hap_thread = threading.Thread(target = haptic_thread, daemon = True, args=())
hap_thread.start()
check_distance_thread = threading.Thread(target = check_distance_ee_al, daemon = True, args=(hap_act_dist,
                                                                                             react_vel_threshold,))
check_distance_thread.start()
            
try:
    with open(user_name +'_all_data.csv',  mode='a') as all_data, open(user_name +'_events.csv', 'a') as events:
        writer_all_data = writer(all_data)
        writer_events = writer(events)
        
        initial_robot_pose = rob.getl()
        pose = initial_robot_pose.copy()
        print("robot tcp is at: ", initial_robot_pose[:3])
        while user_attempts > 0:
            user_attempts -= 1
            print("move the hand to a new position and press esc")
            keyboard.wait('esc')

            direction = device_position_r[:3] - initial_robot_pose[:3]
            dir_norm = direction / np.linalg.norm(direction)

            pose[:3] = device_position_r[:3]  - dir_norm * safe_distance_to_hand
            print("press to move closer to tracker ",pose)

            keyboard.wait('esc')
            writer_active = True
            rob.movel(pose, acc = a, vel = v)
            time.sleep(0.3)
            writer_active = False
            print("writer deactivated")
            distance_triggered = False
            error_triggered = False 
            reaction_triggered = False
            rob.movel(initial_robot_pose, acc = a, vel = v)
            clear_output(wait=True)

except:
        all_data.close()
        events.close()
        client.shutdown(socket.SHUT_RDWR)
        socket.close()
        sys.exit()        

move the hand to a new position and press esc
press to move closer to tracker  [-0.5799043350498009, 0.7420220935730192, 0.5526367320992881, 1.2488441804751143, 2.7570528235608256, 0.43771732571365185]
Hand entered haptic area 0.3941006416085328
trying hap action
User has reacted 0.3016942
hap function executed
writer deactivated


In [4]:
import numpy as np 
import csv
from csv import writer
import pandas as pd
from matplotlib import pyplot as plt


# with open('test_user_events',  mode='') as as events:
#     writer_all_data = writer(all_data)
#     writer_events = writer(events)
#     write_data(writer_all_data,data1_1,data1_2,data1_3)
#     write_data(writer_events,data1_1,data1_2,data1_3)

# df1 = pd.read_csv('test1.cvs',delimiter=",", header=None)
df2 = pd.read_csv(user_name +'_events.csv',delimiter=",", header=None)
all_data = df2.to_numpy()
# print(df1)
print(all_data)
print(all_data.shape)
# fig = plt.figure()
# ax1 = plt.axes()
# i = 780
# j = 867

# line, = ax1.plot(all_data[1000*i:1000*j,3],all_data[1000*i:1000*j,0])
# plt.xlabel('time, s')
# plt.ylabel('distance, m')
# ax1.grid()

# plt.show()


[['Hand entered haptic area' 1.6194501362042563e+18]
 ['User has reacted' 0.9496450999999999]
 ['Hand entered haptic area' 1.6194501521361516e+18]
 ['User has reacted' 0.3016942]]
(4, 2)


In [ ]:
import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output

tag = []
state = []
device_position_r = []
velocity = []

def process_data_from_AL():
    global tag,state,device_position_r,velocity,lock
    print("thread has started")
    while True :         
        incoming_message = c.recv(1024).decode("utf-8") #Collect data from port and decode into  string
        data_start_ix = incoming_message.find("[")
        data_end_ix = incoming_message.find("]")
        data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)

        while(not data_found):
            incoming_message = c.recv(1024).decode("utf-8") #Collect data from port and decode into  string
            data_start_ix = incoming_message.find("[")
            data_end_ix = incoming_message.find("]")
            data_found = (data_start_ix != -1) and (data_end_ix != -1) and (data_start_ix < data_end_ix)
        
        incoming_message = incoming_message[data_start_ix+1 : data_end_ix] 
        incoming_message = incoming_message.replace(',','.')
        list_matrix = incoming_message.split(";")
        state,position,velocity = (list_matrix[1],
                                     np.fromstring(list_matrix[2], dtype=np.float,sep =" "),
                                     np.fromstring(list_matrix[3], dtype=np.float,sep =" "))
        #swap_z 

        position[2] *= -1 
        position_al = np.hstack((position,1))
        device_position_r = tf2robot_new @ position_al
        print("position_al",position_al)
        print("device_position_r",device_position_r)
        clear_output(wait=True)

s = socket.socket()

print('socket created ')
port = 20
s.bind(('127.0.0.1', port)) #bind port with ip address
print('socket binded to port ')
s.listen(5)#listening for connection
print('socket listensing ... ')

c, addr = s.accept() #when port connected
print("\ngot connection from ", addr)
while True:
    process_data_from_AL()

hap_do(hap)
import sys
import keyboard

keyboard.add_hotkey('alt', lambda : print('foobar'))
# keyboard.hook_key('alt', lambda e: print('foobar'))
try:
    while True:
        pass
except:
    pass
finally:
    keyboard.unhook_all_hotkeys()
    sys.exit()
f= open("guru99.txt","w+")


# Custom codein the mix  

import time, serial
import sys
import urx
import numpy as np
import keyboard
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
from IPython.display import clear_output
# from _thread import *
import threading
import time

def get_pose():
    global rob, go, time1
    i = 0
    while go:
        i+=1
        prox = rob.getl()
#         if((time.perf_counter_ns()- time1)/1e9 > 5):
#             rob.stopl()
#         print(prox)
    print(i)
    
# from urx.robotiq_two_finger_gripper import Robotiq_Two_Finger_Gripper

# # rob.set_tcp((0,0,0,0,0,0))
# #     rob.set_payload(0.5, (0,0,0))
go = True
p = threading.Thread(target = get_pose, args = (),daemon = True)
p.start()

l = 0.25
v = 0.25
a = 0.25
time1 = time.perf_counter_ns() 
# pose = rob.getl()
# print("robot tcp is at: ", pose)
# print("absolute move in base coordinate ")
# pose[2] += l
# rob.movel(pose, acc=a, vel=v)
# print("relative move in base coordinate ")
rob.translate((l, 0, 0), acc = a, vel = v)
rob.translate((-l, 0, 0), acc = a, vel = v)

# print("relative move back and forth in tool coordinate")

# rob.translate_tool((-l, 0, 0), acc=a, vel=v)
# rob.translate_tool((l, 0, 0), acc=a, vel=v)

print((time.perf_counter_ns() - time1)/ 1e9 )
go = False
#     rob.close()

In [8]:
# stop = time.time_ns()
print((stop - start)/1e9)

15.3757305
